**Preprocessing 2**

This notebook is in continuation with Preprocessing 1 where we saved complete.csv file, here we will be loading it process it further.

Since in earlier file we had data for movies only upto year 2017, we will be getting data related to movies released in 2019, 2019 and 2020 from wikepedia in this file.

In [9]:
!pip install tmdbv3api

# Step 1 - Importing the librararies

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
sb.set()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Step 2 - Loading complete.csv file , which was saved earlier

In [3]:
complete = pd.read_csv('/content/drive/My Drive/Data Science Projects/dataset_movies/complete.csv')
complete.head()

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,James Cameron CCH Pounder Joel David Moore Wes...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Gore Verbinski Johnny Depp Orlando Bloom Jack ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Sam Mendes Christoph Waltz Rory Kinnear Stepha...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Christopher Nolan Tom Hardy Christian Bale Jos...
4,Andrew Stanton,Daryl Sabara,Samantha Morton,Polly Walker,Action Adventure Sci-Fi,john carter,Andrew Stanton Daryl Sabara Samantha Morton Po...


# Step 3 - Getting data for movies released in 2018, 2019 and 2020 from wikepedia using Web Scrapping

In [4]:
link2018 = 'https://en.wikipedia.org/wiki/List_of_American_films_of_2018'
link2019 = 'https://en.wikipedia.org/wiki/List_of_American_films_of_2019'
link2020 = 'https://en.wikipedia.org/wiki/List_of_American_films_of_2020'

In [5]:
# Getting data from 2nd, 3rd, 4th and 5th table of webpage

df1 = pd.read_html(link2018, header = 0)[2] #Default header is 0 only
df2 = pd.read_html(link2018)[3]
df3 = pd.read_html(link2018)[4]
df4 = pd.read_html(link2018)[5]

df5 = pd.read_html(link2019, header = 0)[3] #Default header is 0 only
df6 = pd.read_html(link2019)[4]
df7 = pd.read_html(link2019)[5]
df8 = pd.read_html(link2019)[6]

df9 = pd.read_html(link2020, header = 0)[3] #Default header is 0 only
df10 = pd.read_html(link2020)[4]
df11 = pd.read_html(link2020)[5]
df12 = pd.read_html(link2020)[6]

df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12], axis = 0)
df.head()

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,JANUARY,5,Insidious: The Last Key,Universal Pictures / Blumhouse Productions / S...,Adam Robitel (director); Leigh Whannell (scree...,[2]
1,JANUARY,5,The Strange Ones,Vertical Entertainment,Lauren Wolkstein (director); Christopher Radcl...,[3]
2,JANUARY,5,Stratton,Momentum Pictures,"Simon West (director); Duncan Falconer, Warren...",[4]
3,JANUARY,10,Sweet Country,Samuel Goldwyn Films,"Warwick Thornton (director); David Tranter, St...",[5]
4,JANUARY,12,The Commuter,Lionsgate / StudioCanal / The Picture Company,Jaume Collet-Serra (director); Byron Willinger...,[6]


In [6]:
df.tail()

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
22,DECEMBER,18,Dune,Warner Bros. Pictures / Legendary Entertainment,Denis Villeneuve (director/screenplay); Jon Sp...,[152]
23,DECEMBER,18,West Side Story,20th Century Studios / Amblin Entertainment,Steven Spielberg (director); Tony Kushner (scr...,[3]
24,DECEMBER,18,Coming 2 America,Paramount Pictures,"Craig Brewer (director); Kenya Barris, Barry W...",[153]
25,DECEMBER,23,The Croods 2,Universal Pictures / DreamWorks Animation,"Joel Crawford (director); Kevin Hageman, Dan H...",[154]
26,DECEMBER,25,News of the World,Universal Pictures,Paul Greengrass (director/screenplay); Luke Da...,[155]


In [7]:
# Dropping some unnecessary columns

df.drop(['Opening','Opening.1','Production company','Ref.'], axis = 1, inplace = True)
df.rename(columns = {'Title':'movie_title'}, inplace = True)

df.head()

,movie_title,Cast and crew
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...
2,Stratton,"Simon West (director); Duncan Falconer, Warren..."
3,Sweet Country,"Warwick Thornton (director); David Tranter, St..."
4,The Commuter,Jaume Collet-Serra (director); Byron Willinger...


# Step 4 - Gettiing Genres info from title column and api from themoviedb.com

In [10]:
from tmdbv3api import TMDb, Movie
import json
import requests

tmdb = TMDb()
tmdb.api_key ='5492165c61b1a21c06eb3a3b578a6339' 

tmdb_movie = Movie()

def get_genres(x):
  genres = []
  string = " "
  
  result = tmdb_movie.search(x)
  movie_id = result[0].id
  response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id, tmdb.api_key))
  data_json = response.json()
  
  if(data_json['genres']):
    
    for i in range (0, len(data_json['genres'])):
      genres.append(data_json['genres'][i]['name'])
      
    return string.join(genres)
  
  else:
    return np.nan

df['genres'] = df['movie_title'].map(lambda x: get_genres(x))

df.head()

,movie_title,Cast and crew,genres
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,Mystery Horror Thriller
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...,Thriller Drama
2,Stratton,"Simon West (director); Duncan Falconer, Warren...",Action Thriller
3,Sweet Country,"Warwick Thornton (director); David Tranter, St...",Drama History Western
4,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,Action Thriller


# Step 5 - Preprocessing Cast and Crew column

In [11]:
def get_director(x):
  
  if " (director)" in x:
    return x.split(" (director)")[0]
  elif(" (directors") in x:
    return x.split(" (directors)")[0]
  else:
    return x.split(" (director/screenplay)")[0]

def get_actor1(x):
  return (x.split('screenplay); ')[-1].split(', ')[0])

def get_actor2(x):
  
  if(len(x.split('screenplay); ')[-1].split(', '))>1):
    return (x.split('screenplay); ')[-1].split(', ')[1])
  else:
    return np.nan

def get_actor3(x):
  
  if(len(x.split('screenplay); ')[-1].split(', '))>2):
    return (x.split('screenplay); ')[-1].split(', ')[2])
  else:
    return np.nan
  
  
df['director_name'] = df['Cast and crew'].map(lambda x: get_director(x))
df['actor_1_name'] = df['Cast and crew'].map(lambda x: get_actor1(x))
df['actor_2_name'] = df['Cast and crew'].map(lambda x: get_actor2(x))
df['actor_3_name'] = df['Cast and crew'].map(lambda x: get_actor3(x))

df.head()

,movie_title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,Mystery Horror Thriller,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...,Thriller Drama,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus
2,Stratton,"Simon West (director); Duncan Falconer, Warren...",Action Thriller,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan
3,Sweet Country,"Warwick Thornton (director); David Tranter, St...",Drama History Western,Warwick Thornton,Bryan Brown,Sam Neill,NaN
4,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,Action Thriller,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson


# Step 6 - Finalizing the data and exporting it to final.csv

In [12]:
final = df.loc[:, ['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

final.dropna(inplace = True)

final['movie_title'] = final['movie_title'].str.lower()

final['comb'] = final['director_name'] + ' ' + final['actor_1_name'] + ' ' + final['actor_2_name'] + ' ' + final['actor_3_name'] + ' ' + final['genres']
final = pd.concat([complete, final], axis = 0 ) # Concating with complete.csv

final.to_csv('final.csv', index = False)


In [13]:
final.head()

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,James Cameron CCH Pounder Joel David Moore Wes...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Gore Verbinski Johnny Depp Orlando Bloom Jack ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Sam Mendes Christoph Waltz Rory Kinnear Stepha...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Christopher Nolan Tom Hardy Christian Bale Jos...
4,Andrew Stanton,Daryl Sabara,Samantha Morton,Polly Walker,Action Adventure Sci-Fi,john carter,Andrew Stanton Daryl Sabara Samantha Morton Po...
